In [1]:
# Package imports
import numpy as np
import matplotlib.pyplot as plt
from testCases_v2 import *
import sklearn
import sklearn.datasets
import sklearn.linear_model
from planar_utils import plot_decision_boundary, sigmoid, load_planar_dataset, load_extra_datasets

%matplotlib inline

np.random.seed(1) # set a seed so that the results are consistent

ModuleNotFoundError: No module named 'testCases_v2'

In [2]:
# Datasets
noisy_circles, noisy_moons, blobs, gaussian_quantiles, no_structure = load_extra_datasets()

datasets = {"noisy_circles": noisy_circles,
            "noisy_moons": noisy_moons,
            "blobs": blobs,
            "gaussian_quantiles": gaussian_quantiles}

### START CODE HERE ### (choose your dataset)
dataset = "noisy_moons"
### END CODE HERE ###

X, Y = datasets[dataset]
X, Y = X.T, Y.reshape(1, Y.shape[0])

# make blobs binary
if dataset == "blobs":
    Y = Y%2

# Visualize the data
plt.scatter(X[0, :], X[1, :], c=Y, s=40, cmap=plt.cm.Spectral);

NameError: name 'load_extra_datasets' is not defined

In [3]:
# Train the logistic regression classifier
clf = sklearn.linear_model.LogisticRegressionCV();
clf.fit(X.T, Y.T);


NameError: name 'sklearn' is not defined

In [4]:
# Plot the decision boundary for logistic regression
plot_decision_boundary(lambda x:clf.predict(x),X,Y)

NameError: name 'plot_decision_boundary' is not defined

In [5]:
LR_predictions = clf.predict(X.T)
predictions_sum = float( np.dot(Y,LR_predictions.T) + np.dot(1-Y, 1-LR_predictions.T) )
precent = predictions_sum/Y.shape[1]*100
print('Accuracy of logistic regression: {} % percentage of correctly labelled datapoints'.format(precent))

NameError: name 'clf' is not defined

In [6]:
def initialize_parameters_noisy_moons(n_x, n_h, n_y):
    W1 = np.random.randn(n_h,n_x)*0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h)*0.01
    b2 = np.zeros((n_y,1))
    features_weights = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return features_weights

In [7]:
def forward_propagation_noisy_moons(X, features_weights):
    
    W1 = features_weights["W1"]
    b1 = features_weights["b1"]
    W2 = features_weights["W2"]
    b2 = features_weights["b2"]
    
    Z1 = np.dot(W1,X) + b1 #output size 4*m
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2,A1) + b2
    A2 = sigmoid(Z2)
    
    activations = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, activations

In [8]:
def compute_cost_noisy_moons(A2, Y):
    m = Y.shape[1] 
    logprobs = np.multiply(np.log(A2),Y)
    logprobs = logprobs + np.multiply(np.log(1-A2),1-Y)
    cost = -1*np.sum(logprobs) / m   
    
    cost = np.squeeze(cost)
    
    return cost
    

In [9]:
def backward_propagation_noisy_moons(features_weights, activations, X, Y):
    W1 = features_weights["W1"]
    W2 = features_weights["W2"]
    A1 = activations['A1']
    A2 = activations['A2']
    
    m = Y.shape[1]
    dZ2 = A2-Y #size dz2: 1*m
    dW2 = np.dot(dZ2,A1.T)/m  #dw2: 1*4
    db2 = np.sum(dZ2,axis=1,keepdims = True)/m #size 1*1
    dZ1 = np.multiply(np.dot(W2.T,dZ2), 1 - np.power(A1, 2))  #size 4*m, 
    dW1 = np.dot(dZ1,X.T)/m #size:4*n
    db1 = np.sum(dZ1,axis=1,keepdims = True)/m #size 4*1
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [10]:
def update_parameters_noisy_moons(features_weights, grads, learning_rate = 1.2):
    W1 = features_weights['W1']
    b1 = features_weights['b1']
    W2 = features_weights['W2']
    b2 = features_weights['b2']
    
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    
    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1;
    W2 = W2 - learning_rate*dW2;
    b2 = b2 - learning_rate*db2;
    
    updated_features_weights = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return updated_features_weights

In [11]:
def nn_model_noisy_moons(X, Y, n_h, num_iterations = 10000, print_cost=False):
    np.random.seed(3)
    n_x = X.shape[0]
    n_y = Y.shape[0]
    
    parameters = initialize_parameters_noisy_moons(n_x,n_h,n_y)
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
   
    for i in range(0, num_iterations):
        A2, activations = forward_propagation_noisy_moons(X, parameters)
        cost = compute_cost_noisy_moons(A2, Y)
        grads = backward_propagation_noisy_moons(parameters, activations, X, Y)
        parameters = update_parameters_noisy_moons(parameters, grads)
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters

In [12]:
def predict_noisy_moon(parameters, X):
    A2, cache = forward_propagation_noisy_moons(X, parameters)  #A2 size will be 1*n
    predictions = A2>0.5
    return predictions

In [13]:
# Build a model with a n_h-dimensional hidden layer
parameters = nn_model_noisy_moons(X, Y, n_h = 4, num_iterations = 10000, print_cost=True)

# Plot the decision boundary
plot_decision_boundary(lambda x: predict_noisy_moon(parameters, x.T), X, Y)
plt.title("Decision Boundary for hidden layer size " + str(4))

NameError: name 'X' is not defined

In [14]:
# Print accuracy
predictions = predict_noisy_moon(parameters, X)
predictions_sum = float( np.dot(Y,predictions.T) + np.dot(1-Y, 1-predictions.T) )
precent = predictions_sum/Y.shape[1]*100
print('Accuracy of logistic regression: {} % percentage of correctly labelled datapoints'.format(precent))


NameError: name 'parameters' is not defined